### Trial 1 : RNN (LSTM) 
can handle sequential data (sentences) effectively
1. Embedding layer
2. LSTM layer (sequence processing)
3. Two dense layers for classification


In [137]:
# IMPORTS
import utils
import keras
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# from keras.models import load_model
# from keras.preprocessing import image
import onnxruntime
import tf2onnx
# import keras2onnx

In [138]:
# %pip install keras2onnx

In [160]:
if torch.cuda.is_available():
    device = torch.device('cuda')

device

device(type='cuda')

#### 1. Load cleaned dataset

In [140]:
import json

def load_data(file_path):
    '''
    This function loads the data from the file_path

    Given the path of the dataset this function reads and returns the intents and the corpus of the dataset

    Args:
        - file_path (str) : path of the dataset

    Returns:
        - unique_intents (list[str]) : list of unique intents in the dataset
        - corpus (list[str]) : list of all the sentences in the dataset
        - corpus_intents (list[str]) : list of intents for each sentence in the dataset
        - responses (list[str]) : list of responses for each intent in the dataset
    '''
    unique_intents = []
    corpus = []
    corpus_intents = []
    # responses = []

    with open(file_path, 'r') as f:
        dataset = json.load(f)

        print(dataset.keys())
        # intents = dataset['intents']

        for intent, values in dataset.items():
            print(intent.lower())
            print(len(values))
            if intent not in unique_intents:
                unique_intents.append(intent)
            for sentence in values:
                corpus.append(utils.clean(sentence))
                corpus_intents.append(intent)

    return unique_intents, corpus, corpus_intents

In [141]:
dataset_path = './intent_detection_dataset/final_intents_dataset.json'
unique_intents, corpus, corpus_intents = load_data(dataset_path)

# print shapes and sizes of the dataset
print('Number of unique intents:', len(unique_intents))
# print('Number of responses:', len(responses))
print('Number of examples:', len(corpus))
print('Number of examples:', len(corpus_intents))

# print samples of the dataset
print('Unique intents:', unique_intents)
# print('Responses:', responses)
print('Samples:', corpus[:5])

data = list(zip(corpus, corpus_intents))

print("Training Sample: ", data[0])

dict_keys(['Variable Declaration', 'Constant Declaration', 'Function Declaration', 'Class Declaration', 'Assignment Operation', 'Conditional Operation', 'For Loop', 'While Loop', 'Array Operation', 'Bitwise Operation', 'Mathematical Operation', 'Membership Operation', 'Casting', 'Input', 'Output', 'Assertion', 'Libraries', 'File System', 'IDE Operation', 'Comment', 'Activate Mouse', 'Activate Interactive', 'Interactive Commands', 'Git Operation'])
variable declaration
240
constant declaration
200
function declaration
90
class declaration
30
assignment operation
80
conditional operation
160
for loop
120
while loop
60
array operation
110
bitwise operation
80
mathematical operation
370
membership operation
160
casting
60
input
50
output
120
assertion
40
libraries
40
file system
150
ide operation
290
comment
50
activate mouse
40
activate interactive
30
interactive commands
220
git operation
80
Number of unique intents: 24
Number of examples: 2870
Number of examples: 2870
Unique intents: ['

#### 2. Tokenizing and Embedding 

In [142]:
# create a tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')

# fit the tokenizer on the corpus -> updates internal vocabulary based on corpus
tokenizer.fit_on_texts(corpus)

# convert the corpus to sequences of integers -> each word is replaced by its index in the vocabulary for each sentence
sequences = tokenizer.texts_to_sequences(corpus)

# pad the sequences to the same length -> add padding tokens to the beginning of each sequence to fit the longest sequence
padded_sequences = keras.preprocessing.sequence.pad_sequences(sequences, padding='pre')

# get the number of unique words (vocabulary size)
vocab_size = len(tokenizer.word_index) + 1

# print the tokenizer properties
print('Vocabulary:', tokenizer.word_index)
print('Vocabulary Size:', vocab_size)
print('Shape of Input Sequence (# of examples, longest sequence length):', padded_sequences.shape)
print('Sample Input Sequence:', padded_sequences[0])

Vocabulary: {'<unk>': 1, 'the': 2, 'and': 3, 'to': 4, 'a': 5, 'is': 6, 'it': 7, 'in': 8, 'name': 9, 'new': 10, 'value': 11, 'variable': 12, 'user': 13, 'with': 14, 'of': 15, 'constant': 16, 'list': 17, 'as': 18, 'assign': 19, 'set': 20, 'equal': 21, 'file': 22, 'whether': 23, 'if': 24, 'for': 25, 'check': 26, 'message': 27, 'than': 28, 'initialize': 29, 'declare': 30, 'make': 31, 'from': 32, 'create': 33, 'loop': 34, 'not': 35, 'or': 36, 'date': 37, 'array': 38, 'get': 39, 'type': 40, 'data': 41, 'code': 42, 'are': 43, 'all': 44, 'line': 45, 'define': 46, 'while': 47, 'bitwise': 48, 'comment': 49, 'under': 50, 'product': 51, 'number': 52, 'step': 53, 'config': 54, 'range': 55, 'store': 56, 'write': 57, 'me': 58, 'allocate': 59, 'power': 60, 'output': 61, 'time': 62, 'that': 63, 'parameters': 64, 'go': 65, 'last': 66, 'less': 67, 'using': 68, 'errors': 69, 'same': 70, 'index': 71, 'show': 72, 'string': 73, 'end': 74, 'has': 75, 'amount': 76, 'algorithm': 77, 'save': 78, 'total': 79, 'la

#### 3. Feature Extraction
map intents to indices

In [143]:
# dictionary that maps each intent to a unique index
intent_to_index = {intent: index for index, intent in enumerate(unique_intents)}

# list for each sentence mapped to its corresponding intent index 
corpus_intent_mapped_to_index = [intent_to_index[intent] for intent in corpus_intents]

# the number of classes to classify a sentence into
number_of_classes = len(intent_to_index)

# convert intent_to_index to index_to_intent 
index_to_intent = {index: intent for intent, index in intent_to_index.items()} 

print(index_to_intent)

# one hot encoding for the intents -> length of each vector is equal to the number of classes
# each sequence in the dataset is represented as a one-hot encoded vector that represents the intent of the sequence
targets = keras.utils.to_categorical(corpus_intent_mapped_to_index, number_of_classes)

print('Categorial vector shape:', targets.shape)

{0: 'Variable Declaration', 1: 'Constant Declaration', 2: 'Function Declaration', 3: 'Class Declaration', 4: 'Assignment Operation', 5: 'Conditional Operation', 6: 'For Loop', 7: 'While Loop', 8: 'Array Operation', 9: 'Bitwise Operation', 10: 'Mathematical Operation', 11: 'Membership Operation', 12: 'Casting', 13: 'Input', 14: 'Output', 15: 'Assertion', 16: 'Libraries', 17: 'File System', 18: 'IDE Operation', 19: 'Comment', 20: 'Activate Mouse', 21: 'Activate Interactive', 22: 'Interactive Commands', 23: 'Git Operation'}
Categorial vector shape: (2870, 24)


In [144]:
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, targets, test_size=0.2, random_state=42, shuffle=True)

train_data = list(zip(x_train, y_train))
test_data = list(zip(x_test, y_test))

print('Number of training examples:', len(train_data))
print('Number of testing examples:', len(test_data))

Number of training examples: 2296
Number of testing examples: 574


#### 4. Building the Model

In [145]:
# the input is the the padded sequences with the target value being the one-hot encoded intents
input_dimenstion = len(unique_intents)

# the output is the one-hot encoded intents
output_dimenstion = targets.shape[1]

print(f"Input Dimension: {input_dimenstion}, Output Dimension: {output_dimenstion}")

# Model description
# The model is a sequential model that consists of:
# 1. An embedding layer that converts the input sequences to dense vectors of fixed size
# 2. A Bidirectional LSTM layer that processes the input sequences in both directions
# 3. A Dense layer with 64 units and ReLU activation function
# 4. A Dropout layer with a dropout rate of 0.5
# 5. A Dense layer with the output dimension and softmax activation function for multi-class classification

# define parameters
epochs = 100

# the embedding dimension is the size of the vector for which each word is represented
# the embedding layer of a neural network, output_dim refers to the size of the dense vectors that the layer will generate for each input token (word). 
# essentially, it is the number of dimensions in which each word will be represented.
embedding_dimension = 128

# lstm units
lstm_units = 64

model = keras.Sequential([
    keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dimension),
    keras.layers.Bidirectional(keras.layers.LSTM(lstm_units, dropout=0.2)),
    keras.layers.Dense(lstm_units, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(output_dimenstion, activation='softmax')
])

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', 'f1_score', 'precision', 'recall'])

model.summary()

Input Dimension: 24, Output Dimension: 24


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

#### 5. Train the Model

In [146]:
model.fit(padded_sequences, targets, epochs=epochs, verbose=1)

Epoch 1/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.1411 - f1_score: 0.0373 - loss: 2.9737 - precision: 0.2006 - recall: 0.0018  
Epoch 2/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.5055 - f1_score: 0.2661 - loss: 1.6194 - precision: 0.7953 - recall: 0.2805
Epoch 3/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7611 - f1_score: 0.5891 - loss: 0.8050 - precision: 0.9134 - recall: 0.6298
Epoch 4/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.8766 - f1_score: 0.7708 - loss: 0.4474 - precision: 0.9388 - recall: 0.7797
Epoch 5/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.9339 - f1_score: 0.8550 - loss: 0.2559 - precision: 0.9638 - recall: 0.8865
Epoch 6/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.9559 - f1_score: 0.9207 - loss: 0.1636 - precision: 0.9726 - recall: 0.9201
Epoch 7/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.9708 - f1_score: 0.9461 - loss: 0.1302 - precision: 0.9784 - recall: 0.9578
Epoc

In [147]:
# x_train.shape

In [148]:
# print("Model Input Shape:", model.input_shape)
# print("X_test shape", x_test.shape)
# print(x_test.dtype)
# print(x_test)

In [149]:
# Run a simple prediction with a smaller subset
# sample_input = x_test[:1]  # Take a single sample
# print("Sample input shape:", sample_input.shape)
# print("Sample input shape:", sample_input.shape)
# sample_prediction = model.predict(sample_input)
# print("Sample prediction:", sample_prediction)

# # Check the model summary
# model.summary()

In [150]:
# try:
#     predictions = model.predict(x_test)
#     print("Predictions shape:", predictions.shape)
# except Exception as e:
#     print("Error during prediction on entire dataset:", e)
# print("Checking for NaNs in x_test:", np.isnan(x_test).any())
# print("Checking for infs in x_test:", np.isinf(x_test).any())
# print("x_test mean:", np.mean(x_test))
# print("x_test std deviation:", np.std(x_test))

In [151]:
# predictions = []
# for test_example in x_test:
#     test_example = test_example.reshape(1, -1)
#     print("Test example shape:", test_example.shape)
#     predictions = model.predict(test_example)

# print(x_test.shape)
# # x_test = x_test.reshape(x_test.shape[0], x_test.shape[1])
# # print(x_test.shape)
# # print(x_test.reshape(1, -1).shape)
# # x_test_final = []
# # for test in x_test:
# #     test = test.reshape(1, -1)
# #     print(test.shape)
# #     x_test_final.append(test)

# # print("X_test_final shape:", x_test_final[0].shape)

# print(x_test[0])
# print(x_test[:1])
# print(x_test[:1].shape)

# print(x_test.shape)

In [152]:
# print(x_test.shape)
# # x_test = x_test.reshape(1, -1)
# predictions = []
# for test in x_test:
#     test = test.reshape(1, -1)
#     print(test.shape)
#     predictions.append(model.predict(test))
# # predictions = model.predict(x_test, verbose=1)

# predicted_intents = [index_to_intent[np.argmax(prediction)] for prediction in predictions]
# true_intents = [index_to_intent[np.argmax(intent)] for intent in y_test]


# accuracy = accuracy_score(true_intents, predicted_intents)
# precision = precision_score(true_intents, predicted_intents, average='weighted')
# recall = recall_score(true_intents, predicted_intents, average='weighted')
# f1 = f1_score(true_intents, predicted_intents, average='weighted')

# # print('Accuracy:', accuracy)
# print('Precision:', precision)
# print('Recall:', recall)
# print('F1 Score:', f1)

In [153]:
# model.evaluate(x_test, y_test)

#### 6. Testing the Model

In [154]:
user_input = input("Sentence:")

test_sequences = tokenizer.texts_to_sequences([user_input])

test_padded_sequences = keras.preprocessing.sequence.pad_sequences(test_sequences, padding='pre')

predictions = model.predict(test_padded_sequences)

predicted_intent_index = np.argmax(predictions)

predicted_intent = index_to_intent[predicted_intent_index]

print(f"Predicted Intent: {predicted_intent}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step
Predicted Intent: Variable Declaration


In [155]:
model.save("./models/intent_detection_model.h5")

In [156]:
# import tensorflow as tf
# import tf2onnx

# # Load your TensorFlow model
# model = tf.keras.models.load_model('models\intent_detection_model.keras')

# # Convert the model to ONNX format
# spec = (tf.TensorSpec((None, *model.input_shape[1:]), tf.float32, name="input"),)
# output_path = "../models/intent_detection.onnx"
# model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=output_path)

# # Save the ONNX model
# with open(output_path, "wb") as f:
#     f.write(model_proto.SerializeToString())


In [157]:
# import tensorflow as tf
# import tf2onnx
# import numpy as np

# # Define or load your Sequential model
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(10, activation='relu', input_shape=(784,)),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

# # Provide a dummy input to the model for shape inference
# spec = (tf.TensorSpec((None, 784), tf.float32, name="input"),)

# # Convert the model to ONNX format
# output_path = "path/to/save/model.onnx"
# # model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=output_path)
# model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=output_path)

# # Save the ONNX model
# with open(output_path, "wb") as f:
#     f.write(model_proto.SerializeToString())


In [159]:
loaded_model = tf.keras.models.load_model('models/intent_detection_model.h5')

input = 'declare variable name'

test_sequences = tokenizer.texts_to_sequences([input])

test_padded_sequences = keras.preprocessing.sequence.pad_sequences(test_sequences, padding='pre')

predictions = loaded_model.predict(test_padded_sequences)

predicted_intent_index = np.argmax(predictions)

predicted_intent = index_to_intent[predicted_intent_index]

print(f"Predicted Intent: {predicted_intent}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step
Predicted Intent: Variable Declaration
